# Assignment 3 
Step Forward Feature Selection: A Practical Example in Python

## 1. Make our imports, load the dataset, and split it into training and testing sets.



In [1]:
# import mlxtend
import mlxtend

# Importing numpy as np  
import pandas as pd

# import os
import os

# Import and use only RandomForestClassifier in sklearn
from sklearn.ensemble import RandomForestClassifier 

# Import and use only the train_test_split from sklearn
from sklearn.model_selection import train_test_split 

# Import accuracy_score as acc 
from sklearn.metrics import accuracy_score as acc 

from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn import metrics



In [2]:
# Read csv file into df
npath=os.path.abspath(os.pardir)+"/Data/Raw/winequality-white.csv"

df=pd.read_csv(npath, sep=";") 

df.head(8)


fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.0              0.27         0.36            20.7      0.045   
1            6.3              0.30         0.34             1.6      0.049   
2            8.1              0.28         0.40             6.9      0.050   
3            7.2              0.23         0.32             8.5      0.058   
4            7.2              0.23         0.32             8.5      0.058   
5            8.1              0.28         0.40             6.9      0.050   
6            6.2              0.32         0.16             7.0      0.045   
7            7.0              0.27         0.36            20.7      0.045   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 45.0                 170.0   1.0010  3.00       0.45   
1                 14.0                 132.0   0.9940  3.30       0.49   
2                 30.0                  97.0   0.9951  3.26       0.44   
3                 47.0                 186.0   0.9956  3.19       0.40   
4                 47.0                 186.0   0.9956  3.19       0.40   
5                 30.0                  97.0   0.9951  3.26       0.44   
6                 30.0                 136.0   0.9949  3.18       0.47   
7                 45.0                 170.0   1.0010  3.00       0.45   

   alcohol  quality  
0      8.8        6  
1      9.5        6  
2     10.1        6  
3      9.9        6  
4      9.9        6  
5     10.1        6  
6      9.6        6  
7      8.8        6

In [3]:
df.shape

(4898, 12)

In [4]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    df.values[:,:-1],
    df.values[:,-1:], 
    # is a slice of the array, which includes all the rows and columns except the 1st column (1st column is 0)
    test_size=0.25, # 25% of the data is used to test the model 
    random_state=42) # Make sure if rerun the codes, the results would be the same

In [5]:
#Return a contiguous flattened array
y_train = y_train.ravel() 
y_test = y_test.ravel()

In [6]:
# Print the result
print('Training dataset shape:', X_train.shape, y_train.shape) 
print('Testing dataset shape:', X_test.shape, y_test.shape)

Training dataset shape: (3673, 11) (3673,)
Testing dataset shape: (1225, 11) (1225,)


## 2. Create and save as a csv descriptive statistics set.

In [7]:
#Create and save as a csv a descriptive statistics set of the wine quality dataset 
# save it as "Description" but using our class convention and the date you computed it
YD_Description_02_15=df.describe()
YD_Description_02_15
dpath=os.path.abspath(os.pardir)+"/Results/YD_Description_02_15.csv"
YD_Description_02_15.to_csv(dpath)

## 3. Define a classifier, a step forward feature selector, and perform our feature selection

In [8]:
## Build RF classifier to use in feature selection
# n_estimators=100 is the number of trees in the forest
# n_jobs is the number of jobs to run in parallel, -1 means using all processors
clf = RandomForestClassifier(n_estimators=100, n_jobs=-1) 

### 1) Using k_features=5 to perform feature selection 

In [9]:
# Build step forward feature selection
sfs1 = sfs(clf,
           k_features=5,
           forward=True,
           floating=False,
           verbose=2,
           scoring='accuracy',
           cv=5)

## Perform SFFS 
# Fit the above-defined feature selector on the "winequality-white" dataset
sfs1 = sfs1.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    6.4s finished

[2023-02-19 21:53:27] Features: 1/5 -- score: 0.4925081095108343[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    5.2s finished

[2023-02-19 21:53:32] Features: 2/5 -- score: 0.5401608926949526[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    4.8s finished

[2023-02-19 21:53:37] Features: 3/5 -- score: 0.6016823296075924[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 

k_features=5: 5 features to be selected  
forward = True and floating = False: for forward selection technique   
verbose: level of verbosity to use in logging   
verbose = 2 :detailed logging including timestamp and cv scores at step   
The scoring argument : the evaluation criterion to be used,uses 'accuracy' for sklearn classifiers   
cv argument :for k-fold cross-validation,here is to set our desired folds to 5   

In [11]:
# Which features?
# The columns at these indexes are those which were selected
feat_cols = list(sfs1.k_feature_idx_)
print(feat_cols)

NameError: name 'sfs1' is not defined

### 2) Report the test accuracy.

In [12]:
# Build full model with selected features
clf = RandomForestClassifier(n_estimators=1000, random_state=42, max_depth=4)
clf.fit(X_train[:, feat_cols], y_train)

# Report the testing accuracy with 5 features
# This model has an accuracy score of 51.6% on the test data
y_train_pred = clf.predict(X_train[:, feat_cols])
print('Training accuracy on selected features: %.3f' % acc(y_train, y_train_pred))

y_test_pred = clf.predict(X_test[:, feat_cols]) # Make predictions for the test set 
print('Testing accuracy on selected features: %.3f' % acc(y_test, y_test_pred)) # View accuracy score

NameError: name 'feat_cols' is not defined

In [13]:
# Build full model on ALL features, for comparison
clf = RandomForestClassifier(n_estimators=1000, random_state=42, max_depth=4)
clf.fit(X_train, y_train)

y_train_pred = clf.predict(X_train)
print('Training accuracy on all features: %.3f' % acc(y_train, y_train_pred))

y_test_pred = clf.predict(X_test)
print('Testing accuracy on all features: %.3f' % acc(y_test, y_test_pred))

Training accuracy on all features: 0.566
Testing accuracy on all features: 0.509


## 2. Using k_features=6 to perform feature selection.

In [14]:
#Modify the code and re-run it using k_features=6
sfs1 = sfs(clf,
           k_features=6,
           forward=True,
           floating=False,
           verbose=2,
           scoring='accuracy',
           cv=5)

## Perform SFFS 
# Fit the above-defined feature selector on the "winequality-white" dataset
sfs1 = sfs1.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:   59.5s finished

[2023-02-19 21:42:45] Features: 1/6 -- score: 0.5004051975013438[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   59.7s finished

[2023-02-19 21:43:44] Features: 2/6 -- score: 0.53879293406736[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   52.9s finished

[2023-02-19 21:44:37] Features: 3/6 -- score: 0.5393408589593875[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 ou

In [15]:
# Which features?
# The columns at these indexes are those which were selected
feat_cols = list(sfs1.k_feature_idx_)
print(feat_cols)

[0, 1, 3, 4, 5, 10]


## 2. Report the test accuracy

In [16]:
 # Build full model with selected features
clf2 = RandomForestClassifier(n_estimators=1000, random_state=42, max_depth=4)
clf.fit(X_train[:, feat_cols2], y_train)

y_train_pred = clf.predict(X_train[:, feat_cols])
print('Training accuracy on selected features: %.3f' % acc(y_train, y_train_pred))

y_test_pred = clf.predict(X_test[:, feat_cols]) # Make predictions for the test set 
print('Testing accuracy on selected features: %.3f' % acc(y_test, y_test_pred)) # View accuracy score

NameError: name 'feat_cols2' is not defined

In [ ]:
# Build full model on ALL features, for comparison
clf2 = RandomForestClassifier(n_estimators=1000, random_state=42, max_depth=4)
clf.fit(X_train, y_train)

y_train_pred2 = clf.predict(X_train)
print('Training accuracy on all features: %.3f' % acc(y_train, y_train_pred))

y_test_pred2 = clf.predict(X_test)
print('Testing accuracy on all features: %.3f' % acc(y_test, y_test_pred))

## 4. Create an online repository and clone your datafile structure
https://github.com/YiouDai/Yiou_Assignment3_02_16.git

## 5. Submit a word document with the link to that repository.